<a href="https://colab.research.google.com/github/sundarp17/sundar_info5731_fall2020/blob/master/Assignment_three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [206]:
# Write your code here

import pandas as pd
df=pd.read_csv('/content/joker_reviews.csv')
df.head()



,User Name,Review
0,MihaVrhunc,"Every once in a while a movie comes, that trul..."
1,lesterarnoldpinto,This is a movie that only those who have felt ...
2,Aman_Goyal,"Truly a masterpiece, The Best Hollywood film o..."
3,logical_guy,Joaquin Phoenix gives a tour de force performa...
4,kdagoulis26,Most of the time movies are anticipated like t...


In [208]:
#cleaning the data
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
sw=stopwords.words('english')
nltk.download('punkt')
nltk.download('wordnet')
from textblob import Word
from textblob import TextBlob
from nltk.stem import PorterStemmer
st=PorterStemmer()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [209]:
df['lower_case']=df['Review'].apply(lambda x:" ".join(x.lower() for x in x.split()))
df['punctuation']=df['lower_case'].apply(lambda  x: " ".join(x for x in x.split() if x not in string.punctuation))
df['special_characters']=df['punctuation'].apply(lambda x:" ".join(x.replace('[#,$,%,@,&]', '') for x in x.split()))
df['stop_words']=df['special_characters'].apply(lambda x:" ".join(x for x in x.split() if x not in words))
df['numbers']=df['stop_words'].apply(lambda x:" ".join(x.replace('\d+', '') for x in x.split()))
df['token']=df['numbers'].apply(lambda x: TextBlob(x).words)
df['stemming']=df['token'].apply(lambda x: " ".join([st.stem(word) for word in x]))
df['lemmatization']=df['stemming'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

df.head()

,User Name,Review,lower_case,punctuation,special_characters,stop_words,numbers,token,stemming,lemmatization
0,MihaVrhunc,"Every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...","every once in a while a movie comes, that trul...","[every, once, in, a, while, a, movie, comes, t...",everi onc in a while a movi come that truli ma...,everi onc in a while a movi come that truli ma...
1,lesterarnoldpinto,This is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,this is a movie that only those who have felt ...,"[this, is, a, movie, that, only, those, who, h...",thi is a movi that onli those who have felt al...,thi is a movi that onli those who have felt al...
2,Aman_Goyal,"Truly a masterpiece, The Best Hollywood film o...","truly a masterpiece, the best hollywood film o...","truly a masterpiece, the best hollywood film o...","truly a masterpiece, the best hollywood film o...","truly a masterpiece, the best hollywood film o...","truly a masterpiece, the best hollywood film o...","[truly, a, masterpiece, the, best, hollywood, ...",truli a masterpiec the best hollywood film of ...,truli a masterpiec the best hollywood film of ...
3,logical_guy,Joaquin Phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,joaquin phoenix gives a tour de force performa...,"[joaquin, phoenix, gives, a, tour, de, force, ...",joaquin phoenix give a tour de forc perform fe...,joaquin phoenix give a tour de forc perform fe...
4,kdagoulis26,Most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,most of the time movies are anticipated like t...,"[most, of, the, time, movies, are, anticipated...",most of the time movi are anticip like thi the...,most of the time movi are anticip like thi the...


In [210]:
#trigrams
from nltk.util import ngrams
trigrams={}
for i in df['lemmatization']:
  words = TextBlob(i).ngrams(3)
  length = len(words)
  if length>0:
    for j in words:
      k = tuple(j)
      if k in trigrams:
        trigrams[k]+=1
      else:
        trigrams[k] = 1

trigrams_10 =  {k: trigrams[k] for k in list(trigrams)[:10]}
trigrams_10




{('a', 'movi', 'come'): 1,
 ('a', 'while', 'a'): 1,
 ('come', 'that', 'truli'): 1,
 ('everi', 'onc', 'in'): 1,
 ('in', 'a', 'while'): 1,
 ('movi', 'come', 'that'): 1,
 ('onc', 'in', 'a'): 1,
 ('that', 'truli', 'make'): 1,
 ('truli', 'make', 'an'): 1,
 ('while', 'a', 'movi'): 1}

In [211]:
#bigram
bigrams={}
for i in df['lemmatization']:
  words = TextBlob(i).ngrams(2)
  length = len(words)
  if length>0:
    for j in words:
      k = tuple(j)
      if k in bigrams:
        bigrams[k]+=1
      else:
        bigrams[k] = 1

bigrams_10 =  {k: bigrams[k] for k in list(bigrams)[:10]}
bigrams_10



{('a', 'movi'): 23,
 ('a', 'while'): 1,
 ('come', 'that'): 1,
 ('everi', 'onc'): 1,
 ('in', 'a'): 20,
 ('movi', 'come'): 1,
 ('onc', 'in'): 1,
 ('that', 'truli'): 1,
 ('truli', 'make'): 1,
 ('while', 'a'): 1}

In [212]:
#unigram
unigrams={}
for i in df['lemmatization']:
  words = TextBlob(i).ngrams(1)
  length = len(words)
  if length>0:
    for j in words:
      k = tuple(j)
      if k in unigrams:
        unigrams[k]+=1
      else:
        unigrams[k] = 1

unigrams_10 =  {k: unigrams[k] for k in list(unigrams)[:10]}
unigrams_10

{('a',): 593,
 ('come',): 14,
 ('everi',): 11,
 ('in',): 200,
 ('make',): 36,
 ('movi',): 280,
 ('onc',): 4,
 ('that',): 214,
 ('truli',): 10,
 ('while',): 16}

In [213]:
#probability
pb={}
for key,val in bigrams.items():
  pb[key]=val/unigrams[tuple((key[0],))]

pb_10 = {k: pb[k] for k in list(pb)[:10]}
pb_10

{('a', 'movi'): 0.0387858347386172,
 ('a', 'while'): 0.0016863406408094434,
 ('come', 'that'): 0.07142857142857142,
 ('everi', 'onc'): 0.09090909090909091,
 ('in', 'a'): 0.1,
 ('movi', 'come'): 0.0035714285714285713,
 ('onc', 'in'): 0.25,
 ('that', 'truli'): 0.004672897196261682,
 ('truli', 'make'): 0.1,
 ('while', 'a'): 0.0625}

In [214]:
#3
#Extract all the noun phrases and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by 
#using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. 
#Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets)

#noun phrases
nltk.download('brown')
np={}
for i in df['lemmatization']:
  words = TextBlob(i).noun_phrases
  for j in words:
    if j in np:
      np[j]+=1
    else:
      np[j] = 1
np_10 ={k: np[k] for k in list(np)[:10]}

#max frequency

mf = max({ i for i in np.values()})
np_10
print('maximum frequency',mf)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
maximum frequency 57


In [ ]:
#relative probability

rp={}
for key, val in np.items():
  rp[key]=val/mf


In [215]:
df_nouns = pd.DataFrame(list(rp.items()),columns = ['nouns','rp']) 
df_nouns.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600
nouns,everi onc,impact joaquin 's,'s brillianc grotesqu haunt,wo n't blink,serious funni moment emot rollercoast sometim,multipl emot popping-up,typic action-riddl,super-hero movi,proper psycholog thriller/drama,truli relat,thi movi,encourag violenc,truli thi movi,thi world,hollywood film,comic book,real ife remark,direct cinematographi music,underappreciated/unrecognized/bulli peopl,show class differ,talent rule,'s believ,multipl joker,bitter way,film show,peopl uncomfort peopl consid thi,perfect film,joaquin phoenix,emot depth,'s imposs,referenc heath ledger 's oscar-win,dark knight,wide consid,definit live-act portray,potenti exce,dark knight 's clown princ,time movi,short way,short joker,time i wa,...,just look,true form,absolut stun,i side,evil charact,honestli u,long time movi,thi deserv,big screen,batman fan,i particularli fond,thi movi wa truli,viscer satisfi crescendo i,joker 's pain wa,palpabl i,oscar pool card,now.p thi movi,super hero movi leav,i wa underwhelm,fact becaus hollywood,magic looong time,bo dy,darth vader 's childhood,joker 's relationship,hi mother,proper boogeyman,overpaid lalaland-schmock wo n't,stupid prequel-explains-it-all-train plea,solid filmmak joker,old roll,unrel scene cheesi taxi-driver-mo,ani impact massiv overact,subtl sinist joker moment,trailer rate,joke heath ledger i,act cinematographi music screenplay costum got...,10/10.oscar worthi performancebi,joaquin phoenix.amaz,todd phillip amaz cinematographi,lawrenc sher
rp,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,1,0.0175439,0.0175439,0.0526316,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0350877,0.210526,0.0175439,0.0175439,0.0175439,0.140351,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,...,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439,0.0175439


# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [216]:
# Write your code here
!pip install texthero
import texthero as hero
df['tfidf'] = hero.tfidf(df['lemmatization'])
df[['lemmatization','tfidf']]




,lemmatization,tfidf
0,everi onc in a while a movi come that truli ma...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,thi is a movi that onli those who have felt al...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,truli a masterpiec the best hollywood film of ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,joaquin phoenix give a tour de forc perform fe...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,most of the time movi are anticip like thi the...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
95,disturb just got out of the movi my humor wa m...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
96,wow i honestli got ta tell you it 's one of th...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
97,i am not a batman fan nor am i particularli fo...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
98,i wa underwhelm to say it friendli but not sur...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [230]:
from sklearn.feature_extraction.text import TfidfVectorizer 

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [st.stem(word) for word in tokens if word not in sw]
  return text

df['non numbers']= df['lemmatization'].apply(lambda x:" ".join(x for x in x.split() if not x.isdigit()))
df['number_string']=df['non numbers'].apply(lambda x:" ".join(x.replace('\d+', '') for x in x.split()))
#print(df['number string'].head(1))
tfidf_vect = TfidfVectorizer()

X_tfidf = tfidf_vect.fit_transform(df['number_string'])

X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
X_tfidf_df.columns =tfidf_vect.get_feature_names()
X_tfidf_df.head()

0    everi onc in a while a movi come that truli ma...
Name: number string, dtype: object


,10,10grade,12i,2nd,76th,aaah,abil,abl,about,abov,absolut,absurd,abus,accept,acclaim,accolad,accompli,accomplish,account,accur,achiev,achiv,across,act,acting,action,activ,actor,actual,ad,adapt,add,addit,adult,advanc,adventur,aesthet,affect,after,afternoonso,...,wit,with,without,witti,wo,woman,wonder,word,work,workplac,world,wors,worst,worth,worthi,would,wow,write,writer,writh,written,wrong,wrote,wtf,ya,yawn,ye,yeah,year,yellow,yesterday,yet,you,young,younger,your,yourself,yr,zazi,zero
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.098235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.178925,0.000000,0.0,0.119477,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061870,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.060935,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.102747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.126423,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.048335,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.132845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.160922,0.0,0.0,0.0,0.0,0.0,0.066422,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.097301,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.119240,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.123696,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [233]:
#query
query=df['number_string'].head(1)
query

0    everi onc in a while a movi come that truli ma...
Name: number_string, dtype: object

In [237]:
#cosine
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

text1=query
cosine=[]
for i in df['number_string']:
  vector1 = text_to_vector(str(text1))
  vector2 = text_to_vector(i)
  cosine.append(get_cosine(vector1, vector2))

df_cosine = pd.DataFrame(cosine,columns=['Cosine similarity'])
df_cosine['text'] = df['number_string']
df_cosine.head()

,Cosine similarity,text
0,0.357295,everi onc in a while a movi come that truli ma...
1,0.282166,thi is a movi that onli those who have felt al...
2,0.227068,truli a masterpiec the best hollywood film of ...
3,0.122637,joaquin phoenix give a tour de forc perform fe...
4,0.183793,most of the time movi are anticip like thi the...


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# The GitHub link of your final csv file

# Link: https://github.com/sundarp17/sundar_info5731_fall2020/blob/master/sentimental_analysis.csv